# ASCENSEUR ET ENTREPÔT AUTOMATIQUE

## Première partie : l'entrepôt

### Question 1

Chaque requête se réalise en 4 temps : il faut d'abord que l'assensseur aille à l'étage de chargement du produit (temps $t_1$), ensuite il charge le produit (temps $\tau$), puis il se déplace vers l'étage de dépot (temps $t_2$) et enfin il déchange le produit (temps $\tau$). On a $t_2 \geq 1$, puisque le produit est déplacé d'au moins un étage.

Une condition nécessaire de stabilité (presque sûre) est $\rho = \frac{2\lambda}{\mu} \leq 1$ (les deux types de demande suivent un processus de poisson d'intensité $\lambda$). Avec les bornes précédentes, on peut en déduire la condition nécessaire : $2\lambda (2\tau + 1) \leq 1$

Sachant que l'étage de dépot ou de chargement (selon si on est en stockage ou récupération) est un tirage aléatoire uniforme sur $[\![1,L]\!]$ et que le point E/S est à l'étage 0, l'espérance de $t_2$ est $\frac{L}{2}$. On a donc la condition nécessaire : $2\lambda (2\tau + \frac{L}{2}) \leq 1$

### Quesion 2

En régime permanent, le nombre moyen de tâches effectuées par unité de temps est nécessairement égal (à un coefficient multiplicateur dépendant de l'unité de temps choisi) à la fréquence d'arrivé des tâches $2\lambda$ (si il est inférieur, les tâches s'accumulent et on est pas en régime permanent tandis que l'inverse est impossible puisque la moyenne empirique d'arrivée des tâches tend vers $2\lambda$ et majore cet indicateur).

La loi de Little nous donne l'équivalence entre les indicateurs (i) et (iii) : le critère (iii) calcule $\bar{q}$, tandis que (iii) est égal à $\bar{\lambda}-- 2\tau - \bar{t_2}$. L'espérance de $t_2$ étant $\frac{T}{2}$, la loi de Little donne : $(iii) = 2\lambda*((i) + 2\tau + \frac{T}{2})$.

### Question 3